# Reproduce HRP paper figures

This notebook performs analyses described in the 

**Manuscript:**

Direct fitting improves the accuracy of the horse radish peroxidase competition assay for peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Simulation and analysis of HRP competition assays

Analysis of Ogusucu 2007 Fig.2 data

Analysis of Manta 2009 Fig.2 data

Plots: Figures 1, 2, 3, S1, and S2

**Requirements:**

Python libraries (see **Imports** below)

Data digitized from from Ogusucu 2007

Data digitized from from Manta 2009


##### Imports

In [ ]:
import os
import copy
import math
import csv

import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d
from lmfit import minimize, Parameters, Parameter, report_fit
from scipy.integrate import solve_ivp
import pandas as pd


##### directories

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]
    
fig_dir = os.path.join(prev_dir,"figures")
if not os.path.isdir(fig_dir): os.mkdir(fig_dir) # ensure dir exists

data_dir = os.path.join(prev_dir,"digitized_data")
if not os.path.isdir(data_dir): os.mkdir(data_dir) # ensure dir exists


##### Matplotlib parameters

In [ ]:
%matplotlib inline

mpl_width = 4.5
mpl_height = 3.5
mpl_dpi = 600
mpl_xlabel_fontsize = "large"
mpl_ylabel_fontsize = "large"
mu = "\u03bc"

mpl_width_squarefig = 3.0
mpl_height_squarefig = 3.0

## Functions

In [ ]:
def do_HRP_exp(mod,
               Prx_init_range,
               noise_scale=0,
               assay_end=0.1,
               data_traces=False,
               assay_points=100):
    """
    Performs a HRP experiment by simulating HRP traces and processing the data.
    Requires "a_factor" as a GLOBAL 
    """
        
    # HRP-only trace (Prx=0)
    trace_HRP_only = do_HRP_trace(mod,"Prx_init",0,noise_scale=noise_scale,end_time=assay_end)
    
    # Assay traces
    vary = "Prx_init"
    if type(data_traces) != bool:     # Simulate at same time points as exp
        traces = list(map(lambda Prx_init,data_trace: do_HRP_trace(mod, 
                                                                    vary,
                                                                    Prx_init,
                                                                    noise_scale=noise_scale,
                                                                    end_time=assay_end,
                                                                    time_points=data_trace[:,0],
                                                                    points=assay_points),
                         Prx_init_range,
                         data_traces[1:]))
    else:
        traces = list(map(lambda Prx_init: do_HRP_trace(mod, 
                                                        vary,
                                                        Prx_init,
                                                        noise_scale=noise_scale,
                                                        end_time=assay_end,
                                                        points=assay_points),
                         Prx_init_range))

    # Calculate fraction inhibition
    f_inhi = list(map(lambda trace: calc_f_inhi(trace,trace_HRP_only,mod),traces))
    f_inhi = np.array((Prx_init_range,f_inhi)).transpose()
    
    # fit rate constant to fraction inhibition
    f_inhi_lm = LinearRegression().fit(f_inhi[:,0].reshape(-1,1),f_inhi[:,1])
    rate_constant = f_inhi_lm.coef_[0]
    
    # Add HRP-only trace to data
    traces.insert(0,trace_HRP_only)
    
    return (traces,f_inhi,f_inhi_lm,rate_constant)

def do_HRP_exp_vary_2(mod,
                        vary_param_name,
                        vary_param_range,
                        Prx_init_range):
    """
    Performs several HRP experiments by varying a parameter, simulating HRP traces and processing the data.
    Requires "a_factor" as a GLOBAL 
    """
    # Data array
    lm_param_data = np.zeros((len(vary_param_range),5))
    
    for count,param_init in enumerate(vary_param_range):
        setattr(mod,vary_param_name,param_init)
        HRP_exp = do_HRP_exp(mod, 
                             Prx_init_range,
                             assay_end=assay_end)

        # Grab data
        lm = HRP_exp[2]
        lm_param_data[count,0] = param_init
        lm_param_data[count,1] = lm.coef_[0] # coef
        lm_param_data[count,2] = (lm.coef_[0]-param_init)/param_init  # overestimation
        lm_param_data[count,3] = -lm.intercept_/lm.coef_[0] # x intercept
        lm_param_data[count,4] = lm.intercept_ # y intercept
        
    return lm_param_data

def do_HRP_trace(mod,
                vary,
                vary_init,
                noise_scale=0,
                time_points=False,
                end_time=0.1,
                points=100):
    """
    Simulates a HRP assay trace.
    Requires "a_factor" as a GLOBAL 
    """
    
    # Set expicit experimental concentration
    vary_orig = getattr(mod,vary)
    setattr(mod,vary,vary_init)
    
    # Do Sim (check if time array was supplied)
    if type(time_points) != bool:
        time_points = time_points
    else:
        time_points = np.linspace(0,end_time,points)
    mod.sim_time = time_points
    mod.Simulate(userinit=1)
    
    # Grab data
    assay_time = mod.sim["Time"]
    assay_absorbance = mod.sim["compound_I"]*a_factor
    noise = np.random.normal(0,noise_scale,mod.sim["compound_I"].shape)
    assay_absorbance = assay_absorbance+noise
    trace = np.array((assay_time,assay_absorbance)).transpose()

    return trace
    
def calc_f_inhi(trace, trace_HRP_only, mod):
    """
    Calculates the fractional inhibition of an HRP trace.
    """
    # Takes the mean of the last 10% of points in order to account for noise
    delta_max = trace_HRP_only[-math.floor(len(trace_HRP_only)/10):,1].mean()
    delta_obs = trace[-math.floor(len(trace)/10):,1].mean()
    k_HRP = getattr(mod,"k_HRP")
    HRP_init = getattr(mod,"HRP_init")
    f_inhi = k_HRP*HRP_init*((delta_max - delta_obs)/delta_obs)
    return f_inhi

def calc_a_factor(e_compound_I=-5.4*10**4,c=10**(-6),l=1):
    """
    c = 10**(-6) # Analysis is in micro Molar
    l = 1 # Light path of reader
    e_compound_I = -5.4*10**4 # HRP breaks down into compound_I which is measured spectrophotometrically
    """
    a_factor = e_compound_I * c * l # A per umol HRP (or Compound_I)
    return a_factor

def res_HRP_exp(params, mod, SH_init_range, data_traces):
    """
    A residual function used for fitting 'k_Prx' in an HRP assay model.
    Simulates HRP assay traces under the same conditions as the data traces and finds the residual.
    """
    # Update model
    mod.k_Prx = params['k_Prx'].value

    # HRP-only trace (Prx=0)
    data_HRP_only = data_traces[0]
    trace_HRP_only = do_HRP_trace(mod,"Prx_init",0,time_points=data_HRP_only[:,0])

    # Assay traces
    vary = "Prx_init"
    sim_traces = list(map(lambda Prx_init,data_trace: do_HRP_trace(mod, 
                                                    vary,
                                                    Prx_init,
                                                    time_points=data_trace[:,0]),
                      SH_init_range,
                      data_traces[1:]))

    sim_traces.insert(0,trace_HRP_only)

    residuals = abs(np.vstack(sim_traces[1:])-np.vstack(data_traces[1:]))
    
    # Check that time points align
    t_array_diff = residuals[:,0]>0.00001
    if True in t_array_diff:
        print("Warning: sim and data time arrays don't align")
        
    return residuals[:,1]


## Initialize model

In [ ]:
class HRP_model:
    """
    Class to set up the HRP assay model.
    """

    def __init__(self, k_Prx=17.0, k_HRP=17.0, H2O2_init=4, Prx_init=5, HRP_init=10, compound_I_init=0):
        self.k_Prx = k_Prx
        self.k_HRP = k_HRP
        self.H2O2_init = H2O2_init
        self.Prx_init = Prx_init
        self.HRP_init = HRP_init
        self.compound_I_init = compound_I_init

    def _dydt(self, t, y, k_HRP, k_Prx):
        # y = (HRP, Prx, H2O2, compound_I)
        v0 = k_HRP * y[0] * y[2]      # v_HRP
        v1 = k_Prx * y[1] * y[2]      # v_Prx
        dHRPdt = -v0
        dPrxdt = -v1
        dH2O2dt = -v0 - v1
        dcompound_Idt = v0
        return [dHRPdt, dPrxdt, dH2O2dt, dcompound_Idt]

    def Simulate(self, time_points=None, userinit=0):
        if userinit == 1:
            assert hasattr(self, 'sim_time'), "Time points must be specified as mod.sim_time"
            time_points = self.sim_time
        y0 = [self.HRP_init, self.Prx_init, self.H2O2_init, self.compound_I_init]
        t_span = (0, time_points.max())
        sol = solve_ivp(
            self._dydt,
            t_span,
            y0,
            t_eval=time_points,
            args=(self.k_HRP, self.k_Prx),
            atol=1e-12,
            rtol=1e-10,
            method="LSODA",
        )
        self.sol = sol
        v0 = self.k_HRP * sol.y[0] * sol.y[2]      # v_HRP
        v1 = self.k_Prx * sol.y[1] * sol.y[2]      # v_Prx
        sim = np.hstack((sol.t.reshape(len(sol.t), 1), sol.y.T, np.vstack((v0, v1)).T))
        self.sim = pd.DataFrame(
            sim, columns=['Time', 'HRP', 'Prx', 'H2O2', 'compound_I', 'v0', 'v1']
        )

In [ ]:
# initialize HRP Prx model
mod_HRP_Prx = HRP_model()

## Fractional inhibition analysis with simulated data (Figure 1a,b,c)

In [ ]:
generic_SH_init_range = np.linspace(mod_HRP_Prx.HRP_init/2.5,mod_HRP_Prx.HRP_init*2,5)


In [ ]:
# Data analysis

# Assay params
assay_end = 0.1
a_factor = calc_a_factor()
SH_init_range = generic_SH_init_range
time_points = np.linspace(0,assay_end,501)

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)

# HRP assay with kHRP of 0.22
low_k_HRP = 0.22
setattr(mod,"k_Prx", low_k_HRP)
HRP_exp_kHRPlow = do_HRP_exp(mod, SH_init_range,assay_end=assay_end)

# HRP assay with kHRP of 17
setattr(mod,"k_Prx", 17)
HRP_exp_kHRP17 = do_HRP_exp(mod, SH_init_range,assay_end=assay_end)

# HRP assay with kHRP of 110
high_k_HRP = 110
setattr(mod,"k_Prx", high_k_HRP)
HRP_exp_kHRPhigh = do_HRP_exp(mod, SH_init_range,assay_end=assay_end)

In [ ]:
# Plot traces

# Assign data
HRP_traces = HRP_exp_kHRP17[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width,h=mpl_height)

axarr.plot(HRP_traces[0][:,0], HRP_traces[0][:,1],color="dimgrey",label="0") 

for count,trace in enumerate(HRP_traces[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
               label=f"{SH_init_range[count]}")

axarr.set_xlabel("Time (s)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("Absorbance (403 nm)",fontsize=mpl_ylabel_fontsize)

axarr.legend(title=f"Prx ({mu}M)",ncols=3,columnspacing=1,loc="upper right")
f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_abs403_vs_time.pdf"),dpi=mpl_dpi)


In [ ]:
# Plot d Absorbance

# Assign data
HRP_traces = HRP_exp_kHRP17[0]
HRP_f_inhi = HRP_exp_kHRP17[1]
dAbs = list(abs(trace[-math.floor(len(trace)/10):,1].mean()) for trace in HRP_traces)
Prx_conc = HRP_f_inhi[:,0].tolist()
Prx_conc.insert(0,0)
HRP_dAbs = np.array(list(zip(Prx_conc,dAbs)))

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

bar_width = HRP_dAbs[:,0].max()/len(HRP_dAbs[:,0])*0.2 

plt.bar(HRP_dAbs[:,0], HRP_dAbs[:,1], width=bar_width, align='center') 

axarr.set_xticks(HRP_dAbs[:,0])
axarr.set_xlabel(f"Prx ({mu}M)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$\Delta$ Absorbance (403 nm)",fontsize=mpl_ylabel_fontsize)

f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_dAbs_vs_Prx.pdf"),dpi=mpl_dpi)

In [ ]:
# Plot fractional inhibition

# Assign data
HRP_f_inhi_kHRPlow = HRP_exp_kHRPlow[1]
HRP_f_inhi_lm_kHRPlow = HRP_exp_kHRPlow[2]
HRP_k_kHRPlow = HRP_exp_kHRPlow[3]

HRP_f_inhi_kHRP17 = HRP_exp_kHRP17[1]
HRP_f_inhi_lm_kHRP17 = HRP_exp_kHRP17[2]
HRP_k_kHRP17 = HRP_exp_kHRP17[3]

HRP_f_inhi_kHRPhigh = HRP_exp_kHRPhigh[1]
HRP_f_inhi_lm_kHRPhigh = HRP_exp_kHRPhigh[2]
HRP_k_kHRPhigh = HRP_exp_kHRPhigh[3]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width,h=mpl_height)

plt1 = axarr.plot(HRP_f_inhi_kHRP17[:,0], HRP_f_inhi_kHRP17[:,1], 
                  "bo", 
                  label=f"17 gives {HRP_k_kHRP17:.3}") 
axarr.plot(np.hstack(([0],HRP_f_inhi_kHRP17[:,0])), 
           HRP_f_inhi_lm_kHRP17.predict(np.hstack(([0],HRP_f_inhi_kHRP17[:,0])).reshape(-1, 1)),
          f"k-")

plt2 = axarr.plot(HRP_f_inhi_kHRPhigh[:,0], HRP_f_inhi_kHRPhigh[:,1], 
                  "go", 
                  label=f"{high_k_HRP} gives {HRP_k_kHRPhigh:.3g}") 
axarr.plot(np.hstack(([0],HRP_f_inhi_kHRPhigh[:,0])), 
           HRP_f_inhi_lm_kHRPhigh.predict(np.hstack(([0],HRP_f_inhi_kHRPhigh[:,0])).reshape(-1, 1)),
          f"k-")

ax2 = axarr.twinx()
plt3 = ax2.plot(HRP_f_inhi_kHRPlow[:,0], HRP_f_inhi_kHRPlow[:,1], 
                "ro", 
                label=f"{low_k_HRP} gives {HRP_k_kHRPlow:.3}") 
ax2.plot(np.hstack(([0],HRP_f_inhi_kHRPlow[:,0])), 
           HRP_f_inhi_lm_kHRPlow.predict(np.hstack(([0],HRP_f_inhi_kHRPlow[:,0])).reshape(-1, 1)),
          f"k-")

axarr.set_xlabel(f"[Prx] ({mu}M)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("$k\mathregular{{_{{HRP}}}}$$\cdot$[HRP]$\cdot$\
($\Delta A_{{max}}-\Delta A_{{obs}}$)/$\Delta A_{{obs}}$",
                 fontsize=mpl_ylabel_fontsize)

ax2.set_ylabel(" ",fontsize=mpl_ylabel_fontsize)

axarr.set_ylim(bottom=0)
axarr.set_xlim(left=0)
ax2.set_ylim(bottom=0,top=10)

## Legend
lns = plt1+plt2+plt3
labs = [l.get_label() for l in lns]
leg = axarr.legend(lns, 
                   labs, 
                   frameon=False, 
                   title=f"input $k\mathregular{{_{{Prx}}}}$ to output $k\mathregular{{_{{Prx}}}}$")
leg._legend_box.align = "left"

f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_f-inhi_vs_Prx_var_kHRP_higher.pdf"),dpi=mpl_dpi)


In [ ]:
# R squared
precision = '.5f'

# kPrx = 0.22
print("kPrx = 0.22")
frac_inhi_list_low = HRP_exp_kHRPlow[1][:,1]
lm_low = HRP_exp_kHRPlow[2]
r2 = format(lm_low.score(np.array(SH_init_range).reshape(-1, 1),frac_inhi_list_low),
            precision)
print(f"R^2 = {r2}")
slope = format(lm_low.coef_[0], precision)
print(f"slope (k_Prx) = {slope}")

# kPrx = kHRP
print("\nkPrx = 17")
frac_inhi_list = HRP_exp_kHRP17[1][:,1]
lm = HRP_exp_kHRP17[2]
r2 = format(lm.score(np.array(SH_init_range).reshape(-1, 1),frac_inhi_list),
            precision)
print(f"R^2 = {r2}")
slope = format(lm.coef_[0], precision)
print(f"slope (k_Prx) = {slope}")

# kPrx = 110
print("\nkPrx = 110")
frac_inhi_list_high = HRP_exp_kHRPhigh[1][:,1]
lm_high = HRP_exp_kHRPhigh[2]
r2 = format(lm_high.score(np.array(SH_init_range).reshape(-1, 1),frac_inhi_list_high),
            precision)
print(f"R^2 = {r2}")
slope = format(lm_high.coef_[0], precision)
print(f"slope (k_Prx) = {slope}")
x_int = format(-lm_high.intercept_/lm_high.coef_[0], precision)


### Mis-estimation (Figure 1d)

In [ ]:
# Data analysis

# Assay params
assay_end = 0.5
a_factor = calc_a_factor()
data_points = 100
kPrx_range = np.linspace(0.22,110,data_points)
SH_init_range=generic_SH_init_range

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)

# Simulations
vary_param_name = "k_Prx"
vary_param_range = kPrx_range
kPrx_In_vs_Out = do_HRP_exp_vary_2(mod,
                                    vary_param_name,
                                    vary_param_range,
                                    SH_init_range)

# Fit linear model to k_Prx_In vs k_Prx_Out
kPrx_In_vs_Out_lm = LinearRegression().fit(kPrx_In_vs_Out[:,0].reshape(-1,1),
                                            kPrx_In_vs_Out[:,1])
kPrx_In_vs_Out_grad = kPrx_In_vs_Out_lm.coef_[0]

In [ ]:
# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(kPrx_In_vs_Out[:,0],
           kPrx_In_vs_Out[:,2]*100,
           linestyle='-')
axarr.plot(kPrx_In_vs_Out[:,0],
           kPrx_In_vs_Out[:,0]*0,
           linestyle = "--",
           color = "k")

axarr.set_xlabel(f"$k\mathregular{{_{{Prx}}}}$ ($\mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$)", 
                 fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("Mis-estimation of $k\mathregular{{_{{Prx}}}}$ (%)",
                 fontsize=mpl_ylabel_fontsize)

f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_kPrxOutOverEst_vs_kPrxIn.pdf"),dpi=mpl_dpi)


### Varying parameters in the fractional inhibition assay (Figure S1)

In [ ]:
data_points = 250
SH_init_range = generic_SH_init_range
a_factor = calc_a_factor()
range_up = 5
range_down = 5

# Simulations with vary k_Prx
mod = copy.deepcopy(mod_HRP_Prx)
vary_param_name = "k_Prx"
vary_param_range = np.linspace(getattr(mod,vary_param_name)/range_down,
                               getattr(mod,vary_param_name)*range_up,
                               data_points)
HRP_vkPrx_lm_param_data = do_HRP_exp_vary_2(mod,
                                            vary_param_name,
                                            vary_param_range,
                                            SH_init_range)

# Simulations with vary k_HRP
mod = copy.deepcopy(mod_HRP_Prx)
vary_param_name = "k_HRP"
vary_param_range = np.linspace(getattr(mod,vary_param_name)/range_down,
                               getattr(mod,vary_param_name)*range_up,
                               data_points)
HRP_vkHRP_lm_param_data = do_HRP_exp_vary_2(mod,
                                            vary_param_name,
                                            vary_param_range,
                                            SH_init_range)

# Simulations with vary HRP
mod = copy.deepcopy(mod_HRP_Prx)
vary_param_name = "HRP_init"
vary_param_range = np.linspace(getattr(mod,vary_param_name)/range_down,
                               getattr(mod,vary_param_name)*range_up,
                               data_points)
HRP_vHRP_lm_param_data = do_HRP_exp_vary_2(mod,
                                            vary_param_name,
                                            vary_param_range,
                                            SH_init_range)

# Simulations with vary H2O2
mod = copy.deepcopy(mod_HRP_Prx)
vary_param_name = "H2O2_init"
vary_param_range = np.linspace(getattr(mod,vary_param_name)/range_down,
                               getattr(mod,vary_param_name)*range_up,
                               data_points)
HRP_vH2O2_lm_param_data = do_HRP_exp_vary_2(mod,
                                            vary_param_name,
                                            vary_param_range,
                                            SH_init_range)

In [ ]:
# Plot analysis

# Assign data
lm_data_vkHRP = HRP_vkHRP_lm_param_data
lm_data_vHRP = HRP_vHRP_lm_param_data
lm_data_vH2O2 = HRP_vH2O2_lm_param_data

# Plot
cols=3
rows=2
f, axarr = plt.subplots(rows, cols)
f.set_size_inches(w=cols*mpl_width_squarefig+1, h=rows*mpl_height_squarefig)

# coef
ax = axarr.flat[0]
ax.plot(lm_data_vkHRP[:,0], lm_data_vkHRP[:,1])
ax.set_ylabel(f"Calculated $k\mathregular{{_{{Prx}}}}$ ($\mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$)",
              fontsize=mpl_ylabel_fontsize)

ax = axarr.flat[1]
ax.plot(lm_data_vHRP[:,0], lm_data_vHRP[:,1])

ax = axarr.flat[2]
ax.plot(lm_data_vH2O2[:,0], lm_data_vH2O2[:,1])

# x int
ax = axarr.flat[3]
ax.plot(lm_data_vkHRP[:,0], lm_data_vkHRP[:,2])
ax.set_ylabel(f"x-intercept ({mu}M Prx)",fontsize=mpl_ylabel_fontsize)
ax.set_xlabel(f"$k\mathregular{{_{{HRP}}}}$ ($\mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$)",
              fontsize=mpl_xlabel_fontsize)

ax = axarr.flat[4]
ax.plot(lm_data_vHRP[:,0], lm_data_vHRP[:,2])
ax.set_xlabel(f"HRP ({mu}M)",fontsize=mpl_xlabel_fontsize)

ax = axarr.flat[5]
ax.plot(lm_data_vH2O2[:,0], lm_data_vH2O2[:,2])
ax.set_xlabel(f"H$_2$O$_2$ ({mu}M)",fontsize=mpl_xlabel_fontsize)

f.tight_layout()

f.savefig(os.path.join(fig_dir,f"hrp_lm_params_vs_mod_params.pdf"),dpi=mpl_dpi)


## Direct fit with simulated data (Figure 2)

In [ ]:
# Data analysis

# Assay params
assay_end = 0.1
a_factor = calc_a_factor()
SH_init_range = generic_SH_init_range

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)

# Generate noisy simulated data
noise_scale = 0.003 # estimated visually 
HRP_exp_noisy = do_HRP_exp(mod, 
                     SH_init_range,
                     noise_scale=noise_scale,
                     assay_end=assay_end)

# Set up lmfit parameter library object with each parameter to be fitted
param_lib = Parameters()
param_lib.add('k_Prx',value=1,min=0.0)

# fit k_Prx to simulated data
fit_kPrx_sim = minimize(res_HRP_exp, param_lib, args=(mod, SH_init_range,HRP_exp_noisy[0]))

# HRP assay with fitted k_Prx
HRP_exp_fit_kPrx_sim = do_HRP_exp(mod, 
                             SH_init_range,
                             assay_end=assay_end)


In [ ]:
report_fit(fit_kPrx_sim)


In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = HRP_exp_noisy[0]
HRP_traces_sim = HRP_exp_fit_kPrx_sim[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width,h=mpl_height)

axarr.plot(HRP_traces_exp[0][:,0],HRP_traces_exp[0][:,1],color="dimgrey",label="0") 
for count,trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
               label=f"{SH_init_range[count]}")

for count,trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
                "k--")
    
axarr.set_xlabel("Time (s)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("Absorbance (403 nm)",fontsize=mpl_ylabel_fontsize)

axarr.legend(title=f"Prx ({mu}M)",ncols=3,columnspacing=1,loc="upper right")
f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_fit_kPrx_to_sim_abs403_vs_time_.pdf"),dpi=mpl_dpi)


In [ ]:
# Data analysis

# Assay params
assay_end = 0.5
a_factor = calc_a_factor()
data_points = 25
kPrx_range = np.linspace(mod_HRP_Prx.k_Prx/2.5,mod_HRP_Prx.k_Prx*2.5,data_points)
kPrx_range = np.linspace(0.22,110,data_points)
SH_init_range=generic_SH_init_range

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)

# Data array
kPrx_In_vs_Out = np.zeros((len(kPrx_range),3))

# HRP experiments
for count,kPrx_In in enumerate(kPrx_range):
    setattr(mod,"k_Prx",kPrx_In)
    HRP_exp = do_HRP_exp(mod, 
                         SH_init_range,
                         assay_end=assay_end)
    
    # fit k_Prx to real data
    param_lib = Parameters()
    param_lib.add('k_Prx',value=1,min=0.0)
    fit_kPrx_Oguf3a = minimize(res_HRP_exp, param_lib, args=(mod,SH_init_range,HRP_exp[0]))
    
    kPrx_Out = fit_kPrx_Oguf3a.params["k_Prx"].value
    kPrx_Out_OverEst = (kPrx_Out-kPrx_In)/kPrx_Out
    kPrx_In_vs_Out[count,0] = kPrx_In
    kPrx_In_vs_Out[count,1] = kPrx_Out
    kPrx_In_vs_Out[count,2] = kPrx_Out_OverEst

# Fit linear model to k_Prx_In vs k_Prx_Out
kPrx_In_vs_Out_lm = LinearRegression().fit(kPrx_In_vs_Out[:,0].reshape(-1,1),
                                           kPrx_In_vs_Out[:,1])
kPrx_In_vs_Out_grad = kPrx_In_vs_Out_lm.coef_[0]

In [ ]:
# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(kPrx_In_vs_Out[:,0],kPrx_In_vs_Out[:,1],"o",label="obtained")
axarr.plot(kPrx_In_vs_Out[:,0],kPrx_In_vs_Out[:,0],"k--",label="identical")

axarr.set_xlabel(f"Input $k\mathregular{{_{{Prx}}}}$ ($\mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$)", 
                 fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f"Determined $k\mathregular{{_{{Prx}}}}$ ($\mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$)",
                 fontsize=mpl_ylabel_fontsize)

axarr.legend()
f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_kPrxIn_vs_kPrxOut_byODE.pdf"),dpi=mpl_dpi)


## Analysis of digitized published data

### Ogusucu 2007 (Figure 3)

In [ ]:
# Load experimental data
Oguf3a_data_dir = "Ogusucu_2007_Fig2"
files_in_directory = os.listdir(os.path.join(data_dir,Oguf3a_data_dir))
files_in_directory.sort() # Prx=0 will be in pos 0

Oguf3a_traces = list()
Oguf3a_Prx_init_range = list()
for filename in files_in_directory:
    if filename[-3:] == "csv": # excludes non .csv files
        with open(os.path.join(os.path.join(data_dir,Oguf3a_data_dir),filename),"r") as csv_file:
            csv_gen = (line.replace(",", ".") for line in csv_file) # change comma decimels to period
            trace = np.genfromtxt(csv_gen, delimiter=";")
            trace = trace[trace[:, 0].argsort()]
            Oguf3a_traces.append(trace)

            # Get Prx init
            Oguf3a_Prx_init = filename[-7:-4]
            Oguf3a_Prx_init_range.append(float(Oguf3a_Prx_init))

Oguf3a_Prx_init_range = Oguf3a_Prx_init_range[1:] # Remove 0.0
Oguf3a_Prx_init_range = np.array(Oguf3a_Prx_init_range)


In [ ]:
# Data analysis frac inhi

# Assay params
HRP_oguf3a = 3.67 # μM
H2O2_oguf3a = 1.5 # μM
assay_end = 0.1
l_oguf3a = 0.85
a_factor = calc_a_factor(l=l_oguf3a)
kPrx_pub_oguf3a = 13.0 # uM−1.s−1

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)
setattr(mod,"HRP_init",HRP_oguf3a)
setattr(mod,"H2O2_init",H2O2_oguf3a)

# This is the same as in do_HRP_exp except the traces are imported
traces = Oguf3a_traces[1:]
trace_HRP_only = Oguf3a_traces[0]

# Calculate fraction inhibition
f_inhi = list(map(lambda trace: calc_f_inhi(trace,trace_HRP_only,mod),traces))
f_inhi = np.array((Oguf3a_Prx_init_range,f_inhi)).transpose()

# fit rate constant to fraction inhibition
f_inhi_lm = LinearRegression().fit(f_inhi[:,0].reshape(-1,1),f_inhi[:,1])
rate_constant = f_inhi_lm.coef_[0]

# Add HRP-only trace to data
traces.insert(0,trace_HRP_only)

Oguf3a_exp = (traces,f_inhi,f_inhi_lm,rate_constant)


In [ ]:
# Data analysis ODE

# Assay params
HRP_oguf3a = 3.67 # μM
H2O2_oguf3a = 1.5 # μM
assay_end = 0.1
l_oguf3a = 0.85
a_factor = calc_a_factor(l=l_oguf3a)
kPrx_pub_oguf3a = 13.0 # uM−1.s−1

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)
setattr(mod,"HRP_init",HRP_oguf3a)
setattr(mod,"H2O2_init",H2O2_oguf3a)

# HRP assay with published k_Prx
setattr(mod,"k_Prx",kPrx_pub_oguf3a)
HRP_exp_pub_kPrx_Oguf3a = do_HRP_exp(mod, 
                                     Oguf3a_Prx_init_range,
                                     data_traces=Oguf3a_traces,
                                     assay_end=assay_end)

# fit k_Prx to real data
param_lib = Parameters()
param_lib.add('k_Prx',value=1,min=0.0)
fit_kPrx_Oguf3a = minimize(res_HRP_exp, param_lib, args=(mod,
                                                         Oguf3a_Prx_init_range,
                                                         Oguf3a_traces))

# HRP assay with fitted k_Prx
HRP_exp_fit_kPrx_Oguf3a = do_HRP_exp(mod, 
                             Oguf3a_Prx_init_range,
                             data_traces=Oguf3a_traces,
                             assay_end=assay_end)


In [ ]:
report_fit(fit_kPrx_Oguf3a)


#### Plot model fit with published $k_{Prx}$ of 13 µM<sup>-1</sup>s<sup>-1</sup>

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = Oguf3a_traces
HRP_traces_sim = HRP_exp_pub_kPrx_Oguf3a[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width,h=mpl_height)

axarr.plot(HRP_traces_exp[0][:,0],HRP_traces_exp[0][:,1],color="dimgrey",label="0") 
for count,trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
               label=f"{SH_init_range[count]}")

axarr.plot(HRP_traces_sim[0][:,0],HRP_traces_sim[0][:,1],"k--") 
for count,trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
                "k--")
    
axarr.set_xlabel("Time (s)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("Absorbance (403 nm)",fontsize=mpl_ylabel_fontsize)

axarr.legend(title=f"Prx ({mu}M)",ncols=3,columnspacing=1,loc="upper right")
f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_Ogu_pub_kPrx_abs403_vs_time.pdf"),dpi=mpl_dpi)

# chi square
chi_square_ogu = sum((np.vstack(HRP_traces_sim[1:])[:,1]
                      -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_ogu)


#### Plot model fit with direct fitted $k_{Prx}$ of 11.18 µM<sup>-1</sup>s<sup>-1</sup>

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = Oguf3a_traces
HRP_traces_sim = HRP_exp_fit_kPrx_Oguf3a[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width,h=mpl_height)

axarr.plot(HRP_traces_exp[0][:,0],HRP_traces_exp[0][:,1],color="dimgrey",label="0") 
for count,trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
               label=f"{SH_init_range[count]}")

axarr.plot(HRP_traces_sim[0][:,0],HRP_traces_sim[0][:,1],"k--") 
for count,trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:,0], 
               trace[:,1], 
                "k--")
    
axarr.set_xlabel("Time (s)", fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel("Absorbance (403 nm)",fontsize=mpl_ylabel_fontsize)

axarr.legend(title=f"Prx ({mu}M)",ncols=3,columnspacing=1,loc="upper right")
f.tight_layout()

f.savefig(os.path.join(fig_dir,"hrp_Oguf3a_fit_kPrx_abs403_vs_time.pdf"),dpi=mpl_dpi)

# chi square
chi_square_ogu = sum((np.vstack(HRP_traces_sim[1:])[:,1]
                      -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_ogu)


#### Mis-estimation

In [ ]:
# Data analysis

# Assay params
assay_end = 0.5
a_factor = calc_a_factor()
data_points = 100
kPrx_range = np.linspace(0.22,110,data_points)
SH_init_range=generic_SH_init_range

# Setup model
mod = copy.deepcopy(mod_HRP_Prx)
setattr(mod,"HRP_init",HRP_oguf3a)
setattr(mod,"H2O2_init",H2O2_oguf3a)

# Simulations with vary k_Prx
vary_param_name = "k_Prx"
vary_param_range = kPrx_range
kPrx_Oguf3a_In_vs_Out = do_HRP_exp_vary_2(mod,
                                    vary_param_name,
                                    vary_param_range,
                                    SH_init_range)

# Fit linear model to k_Prx_In vs k_Prx_Out
kPrx_Oguf3a_In_vs_Out_lm = LinearRegression().fit(
    kPrx_Oguf3a_In_vs_Out[:,0].reshape(-1,1),
    kPrx_Oguf3a_In_vs_Out[:,1])
kPrx_Oguf3a_In_vs_Out_grad = kPrx_Oguf3a_In_vs_Out_lm.coef_[0]

# fit kPrx in Ogusucu to overestimation
idx = np.searchsorted(kPrx_range, kPrx_pub_oguf3a, side="left")
x_points_fit = [kPrx_Oguf3a_In_vs_Out[:,0][idx-1], kPrx_Oguf3a_In_vs_Out[:,0][idx]]
y_points_fit = [kPrx_Oguf3a_In_vs_Out[:,2][idx-1], kPrx_Oguf3a_In_vs_Out[:,2][idx]]
lm = interp1d(x_points_fit, y_points_fit)
y_intrepolated = lm(kPrx_pub_oguf3a)

In [ ]:
k_Prx_fitted = fit_kPrx_Oguf3a.params["k_Prx"].value
k_Prx_overestimation = (kPrx_pub_oguf3a/k_Prx_fitted - 1)*100
print(f"l_oguf3a: {l_oguf3a} cm")
print(f"k_Prx_fitted: {str(round(k_Prx_fitted,2))}")
print(f"k_Prx_overestimation: {str(k_Prx_overestimation)[:4]} %")
print(f"predicted_overestimation: {str(lm(kPrx_pub_oguf3a)*100)[:4]} %")


### Manta 2009 (Figure S2)

In [ ]:
# Load experimental data
Mantaf2a_data_dir = "Manta_2009_Fig2"
files_in_directory = os.listdir(os.path.join(data_dir,Mantaf2a_data_dir))
files_in_directory.sort() # Prx=0 will be in pos 0

# Name
fig_name_suffix = "Manta"

# Assay parameters
Mantaf2a_Prx_init_range = list()
for filename in files_in_directory:
    if filename[-3:] == "csv": # excludes non .csv files
        Mantaf2a_Prx_init = filename[-10:-7]
        Mantaf2a_Prx_init_range.append(float(Mantaf2a_Prx_init))

exp_Prx_initials = Mantaf2a_Prx_init_range
exp_Prx_initials = [float(Prx) for Prx in exp_Prx_initials]
exp_HRP_initial = 5.0
exp_H2O2_initial = 1.0

# Method parameters
exp_kHRP = 11.0
exp_Abs_nm = "398"
exp_lightpath = 0.78
e_compound_I_Abs_nm = -54000

# Data traces
Mantaf2a_traces = list()
Mantaf2a_Prx_init_range = list()
for filename in files_in_directory:
    if filename[-3:] == "csv": # excludes non .csv files
        with open(os.path.join(os.path.join(data_dir,Mantaf2a_data_dir),filename),"r") as csv_file:
            trace = np.genfromtxt(csv_file, delimiter=",")
            trace = trace[trace[:, 0].argsort()]
            Mantaf2a_traces.append(trace)
exp_HRP_traces = Mantaf2a_traces


In [ ]:
a_factor = calc_a_factor(e_compound_I=e_compound_I_Abs_nm, l=exp_lightpath)

In [ ]:
# Initialize model
mod_HRP_Manta2009 = HRP_model()

setattr(mod_HRP_Manta2009, 'HRP_init', exp_HRP_initial)
setattr(mod_HRP_Manta2009, 'H2O2_init', exp_H2O2_initial)
setattr(mod_HRP_Manta2009, 'k_HRP', exp_kHRP)

#### Direct fit to each trace individually

In [ ]:
# Model setup
mod_direct_indiv = copy.deepcopy(mod_HRP_Manta2009)

In [ ]:
# Fit
direct_fit_indiv_kPrxs = []
direct_fit_indiv_stderr = []

for count, exp_HRP_trace in enumerate(exp_HRP_traces[1:]):
    # setup param lib
    param_lib = Parameters()
    param_lib.add('k_Prx', value=1, min=0.0)

    # do the fit
    fit_kPrx = minimize(
        res_HRP_exp,
        param_lib,
        args=(
            mod_direct_indiv,
            [exp_Prx_initials[1:][count]],
            [exp_HRP_traces[0], exp_HRP_trace],
        ),
    )

    # store results
    direct_fit_indiv_kPrxs.append(fit_kPrx.params['k_Prx'].value)
    direct_fit_indiv_stderr.append(fit_kPrx.params['k_Prx'].stderr)


In [ ]:
# Display fitted values
print(direct_fit_indiv_kPrxs)

In [ ]:
# Simulate data
sim_HRP_traces = []
vary = "Prx_init"

for count, kPrx in enumerate(direct_fit_indiv_kPrxs):
    setattr(mod_direct_indiv, 'k_Prx', kPrx)

    HRP_trace = do_HRP_trace(
        mod_direct_indiv,
        vary,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_direct_ind-fit_traces_{fig_name_suffix}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum((np.vstack(HRP_traces_sim)[:,1]
                            -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_dir_indiv)

#### Direct fit to all traces simultaneously

In [ ]:
# Model setup
mod_direct_all = copy.deepcopy(mod_HRP_Manta2009)

In [ ]:
# Fit 
param_lib = Parameters()
param_lib.add('k_Prx', value=1, min=0.0)

# do the fit
fit = minimize(
    res_HRP_exp, param_lib, args=(mod_direct_all, exp_Prx_initials[1:], exp_HRP_traces)
)

# store results
direct_fit_whole_kPrx = fit.params['k_Prx'].value
direct_fit_whole_stderr = fit.params['k_Prx'].stderr

In [ ]:
report_fit(fit)

In [ ]:
# Simulate traces with fitted k_Prx
setattr(mod_direct_all, 'k_Prx', direct_fit_whole_kPrx)
vary = "Prx_init"

sim_HRP_traces = []

for count, SH_init in enumerate(exp_Prx_initials[1:]):

    HRP_trace = do_HRP_trace(
        mod_direct_all,
        vary,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_direct_wh-fit_traces_{fig_name_suffix}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum((np.vstack(HRP_traces_sim)[:,1]
                            -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_dir_indiv)

#### Fractional inhibition analysis on all traces

In [ ]:
# Model setup
mod_finhi_all = copy.deepcopy(mod_HRP_Manta2009)

In [ ]:
# Plot analysis

# Assign data
HRP_traces = exp_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces[0][:, 0], HRP_traces[0][:, 1], color='dimgrey', label='0')

for count, trace in enumerate(HRP_traces[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()


In [ ]:
# Plot analysis

# Assign data
HRP_traces = exp_HRP_traces

dAbs = list(
    abs(trace[-math.floor(len(trace) / 10) :, 1].mean()) for trace in HRP_traces
)

HRP_dAbs = np.array(list(zip(exp_Prx_initials, dAbs)))

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

bar_width = (
    HRP_dAbs[:, 0].max() / len(HRP_dAbs[:, 0]) * 0.2
)   # set bar_width to a fraction of space per data point

plt.bar(HRP_dAbs[:, 0], HRP_dAbs[:, 1], width=bar_width, align='center')

axarr.set_xticks(HRP_dAbs[:, 0])
axarr.set_xlabel(f'Prx ({mu}M)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'$\Delta$ Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)

f.tight_layout()

f.savefig(os.path.join(fig_dir, f'HRP_f-inhi_dAbs_{fig_name_suffix}.pdf'), dpi=mpl_dpi)

In [ ]:
# Plot analysis
HRP_traces = exp_HRP_traces

# Assign data
HRP_f_inhi = list(
    map(lambda trace: calc_f_inhi(trace, HRP_traces[0], mod_finhi_all), HRP_traces[1:])
)
HRP_f_inhi = np.array((exp_Prx_initials[1:], HRP_f_inhi)).transpose()
HRP_f_inhi_lm = LinearRegression().fit(
    HRP_f_inhi[:, 0].reshape(-1, 1), HRP_f_inhi[:, 1]
)
frac_inhi_whole_kPrx = HRP_f_inhi_lm.coef_[0]

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(
    HRP_f_inhi[:, 0],
    HRP_f_inhi[:, 1],
    'bo',
    label=f'$k_{{Prx}} = {frac_inhi_whole_kPrx:.3}\ \mathrm{{ µM^{{{-1}}}\cdot s^{{{-1}}} }}$',
)
axarr.plot(
    np.hstack(([0], HRP_f_inhi[:, 0])),
    HRP_f_inhi_lm.predict(np.hstack(([0], HRP_f_inhi[:, 0])).reshape(-1, 1)),
    f'k-',
)

axarr.set_xlabel(f'[Prx] ({mu}M)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(
    '$k\mathregular{{_{{HRP}}}}$$\cdot$[HRP]$\cdot$\
($\Delta A_{{{max}}}-\Delta A_{{{obs}}}$)/$\Delta A_{{{obs}}}$',
    fontsize=mpl_ylabel_fontsize,
)

axarr.set_ylim(bottom=0)
axarr.set_xlim(left=0)
axarr.legend()

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_f-inhi_vs_Prx_{fig_name_suffix}.pdf'), dpi=mpl_dpi
)

In [ ]:
# Simulate data

setattr(mod_finhi_all, 'k_Prx', frac_inhi_whole_kPrx)

sim_HRP_traces = []
vary = "Prx_init"

for count, SH_init in enumerate(exp_Prx_initials[1:]):

    HRP_trace = do_HRP_trace(
        mod_finhi_all,
        vary,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_f-inhi_wh-fit_traces_{fig_name_suffix}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum((np.vstack(HRP_traces_sim)[:,1]
                            -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_dir_indiv)

#### Fractional inhibition analysis on each trace separately

In [ ]:
# Model setup
mod_finhi_indiv = copy.deepcopy(mod_HRP_Manta2009)

In [ ]:
frac_inhi_indiv_kPrxs = HRP_f_inhi[:, 1] / HRP_f_inhi[:, 0] - HRP_f_inhi_lm.intercept_
print(f'fitted k_Prxs: {frac_inhi_indiv_kPrxs}')

In [ ]:
# Simulate data
sim_HRP_traces = []
vary = "Prx_init"

for count, kPrx in enumerate(frac_inhi_indiv_kPrxs):
    setattr(mod_finhi_indiv, 'k_Prx', kPrx)

    HRP_trace = do_HRP_trace(
        mod_finhi_indiv,
        vary,
        exp_Prx_initials[1:][count],
        time_points=exp_HRP_traces[1:][count][:, 0],
    )

    sim_HRP_traces.append(HRP_trace)

In [ ]:
# Plot analysis

# Assign data
HRP_traces_exp = exp_HRP_traces
HRP_traces_sim = sim_HRP_traces

# Plot
f, axarr = plt.subplots(1, 1)
f.set_size_inches(w=mpl_width, h=mpl_height)

axarr.plot(HRP_traces_exp[0][:, 0], HRP_traces_exp[0][:, 1], color='dimgrey', label='0')
for count, trace in enumerate(HRP_traces_exp[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], label=f'{exp_Prx_initials[1:][count]}')

axarr.plot(HRP_traces_sim[0][:, 0], HRP_traces_sim[0][:, 1], 'k--')
for count, trace in enumerate(HRP_traces_sim[1:]):
    axarr.plot(trace[:, 0], trace[:, 1], 'k--')

axarr.set_xlabel('Time (s)', fontsize=mpl_xlabel_fontsize)
axarr.set_ylabel(f'Absorbance ({exp_Abs_nm} nm)', fontsize=mpl_ylabel_fontsize)
axarr.legend(title=f'Prx ({mu}M)', ncols=3, columnspacing=1, loc='upper right')

f.tight_layout()

f.savefig(
    os.path.join(fig_dir, f'HRP_f-inhi_ind-fit_traces_{fig_name_suffix}.pdf'),
    dpi=mpl_dpi,
)

# chi square
chi_square_dir_indiv = sum((np.vstack(HRP_traces_sim)[:,1]
                            -np.vstack(HRP_traces_exp[1:])[:,1])**2)

print(chi_square_dir_indiv)